Accessing OpenSky Network token for sending API requests

In [ ]:
import requests
import json
from time import time
import pandas as pd
import logging

credentials_file_path = "opensky_credentials.json"

try:
    with open(credentials_file_path, 'r', encoding='utf-8') as f:
        credentials = json.load(f)
        logging.info("Successfully loaded credentials attributes.")
except FileNotFoundError:
    logging.error(f"Error: The file '{credentials_file_path}' was not found. Please check the path.")
except json.JSONDecodeError:
    logging.error(f"Error: The file '{credentials_file_path}' is not valid JSON.")
except Exception as e:
    logging.error(f"An unexpected error occurred: {e}")

# 1. Your OpenSky API Client Credentials
CLIENT_ID = credentials['clientId']
CLIENT_SECRET = credentials['clientSecret']

AUTH_URL = "https://auth.opensky-network.org/auth/realms/opensky-network/protocol/openid-connect/token"
API_BASE_URL = "https://opensky-network.org/api"

def get_access_token(client_id, client_secret):
    """Requests a new access token from the OpenSky auth server."""
    logging.info("Requesting new Access Token...")
    
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }
    
    data = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }
    
    try:
        response = requests.post(AUTH_URL, headers=headers, data=data)
        response.raise_for_status()  
        token_data = response.json()
        
        # The token is valid for 'expires_in' seconds (usually 1800 seconds or 30 minutes)
        access_token = token_data.get("access_token")
        #expires_in = token_data.get("expires_in", 1800)
        
        logging.info(f"Successfully retrieved Access Token.")
        return access_token
        
    except requests.exceptions.RequestException as e:
        logging.error(f"Error requesting token: {e}")
        return None, 0

Function to send request to arrivals, departures, and aircrafts.

In [49]:
def make_OpenSky_request(API_BASE_URL, endpoint, param, airport_or_icao24_value, begin_ts, end_ts, token):
    """Makes an API request using the Bearer Token."""
    if not token:
        logging.error("Error: No valid token available.")
        raise "notValidTokenError"
        
    url = f"{API_BASE_URL}{endpoint}"
    logging.info(f"\nMaking API request to {url}...")
    
    # As this function works for two REST API endpoints, depending on the param provided the params were selected  
    if param == "airport":
        params = {
            "airport": airport_or_icao24_value,
            "begin": begin_ts,
            "end": end_ts
        }
    else:
        params = {
            "icao24": airport_or_icao24_value,
            "begin": begin_ts,
            "end": end_ts
        }
    
    logging.info(f"Param:'{param}' is selected, therefore params: {params}")
    
    headers = {
        "Authorization": f"Bearer {token}"
    }
    
    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status() 
        return response
        
    except requests.exceptions.RequestException as e:
        logging.error(f"Error making API request: {e}")
        raise e

In [ ]:
# Retrieve the token
token = get_access_token(CLIENT_ID, CLIENT_SECRET)
  
MAX_RETRIES = 3
current_try = 0

while current_try <= MAX_RETRIES:  
    
    response = make_OpenSky_request(API_BASE_URL, "/flights/aircraft", "icao24", "4BCDEC", 1735820000, 1735941600, token)

    if response.status_code == 200:
        data = response.json()
        logging.info(f"Successfully retrieved Aircraft vector records.")
        df = pd.DataFrame(data)
    elif response.status_code == 401:
        logging.warn("Token might have expired. Sending request to get new token...")
        # Retrieve the token
        token = get_access_token(CLIENT_ID, CLIENT_SECRET)
        
        current_try += 1
    elif response.status_code > 401:
        logging.error(f"Error while retrieving the data. Status Code: {response.status_code}")
        raise "unknownError"

Extracting Arrivals and Departures from AeroDataBox API

In [ ]:
import requests
import urllib.parse

aerodatabox_api_key_file_path = "aerodatabox_api_key.json"

try:
    with open(aerodatabox_api_key_file_path, 'r', encoding='utf-8') as f:
        credentials = json.load(f)
        logging.info("Successfully loaded credentials attributes.")
except FileNotFoundError:
    logging.error(f"Error: The file '{aerodatabox_api_key_file_path}' was not found. Please check the path.")
except json.JSONDecodeError:
    logging.error(f"Error: The file '{aerodatabox_api_key_file_path}' is not valid JSON.")
except Exception as e:
    logging.error(f"An unexpected error occurred: {e}")


delays_base_url = "https://prod.api.market/api/v1/aedbx/aerodatabox"
codetype = "icao"
code = "KJFK"
endpoint = f"flights/airports/{codetype}/{code}"
TIME_FROM = "2025-11-02T00:00"
TIME_TO = "2025-11-02T01:00"

# 1. URL-encode the time strings
encoded_from = urllib.parse.quote(TIME_FROM)
encoded_to = urllib.parse.quote(TIME_TO)

headers = {
    "accept": "application/json",
    "x-api-market-key": credentials['key'],
}

# Combine the base URL and the endpoint for the final request URL
full_url = f"{delays_base_url}/{endpoint}/{encoded_from}/{encoded_to}"

try:
    # Use the full_url for the request
    response = requests.get(full_url, headers=headers)
    
    # Check for HTTP errors before trying to parse JSON
    response.raise_for_status() 
    
    print(response.json())
    
except requests.exceptions.HTTPError as errh:
    logging.error(f"Http Error: {errh}")
except requests.exceptions.ConnectionError as errc:
    logging.error(f"Error Connecting: {errc}")
except requests.exceptions.Timeout as errt:
    logging.error(f"Timeout Error: {errt}")
except requests.exceptions.RequestException as e:
    # This catches the original exception and others not caught above
    logging.error(f"An unexpected API request error occurred: {e}")
except Exception as e:
    # Catches non-request errors, like JSON decoding failure
    logging.error(f"An unexpected error occurred: {e}")